# TensorFlow Tutorials
# Load and Preprocess Data 01 - CSV

Using the `titanic` dataset from Kaggle to demonstrate how to load and preprocess data from CSV (comma-separated value) files using `tensorflow` `csv_dataset` structures.

In [0]:
try:
  # %tensorflow_version only exists in CoLab
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
# functools is a library that allows us to use higher order functions
# these are functions that can both take other functions as params and return them
import functools

In [0]:
import numpy as np
import tensorflow as tf

# Checking version
print(tf.__version__)

2.0.0-rc1


## Loading Data
Data is stored as CSV files on Google's CDN. It will first be downloaded to the storage linked to this CoLab notebook, and will then be read into a `tf.dataset` structure using `tf.experimental.make_csv_dataset` function.

In [0]:
# URLs for CSV files
TRAIN_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/train.csv"
TEST_DATA_URL = "https://storage.googleapis.com/tf-datasets/titanic/eval.csv"

In [0]:
# Store filepaths for these CSVs
train_file_path = tf.keras.utils.get_file("train.csv", TRAIN_DATA_URL)
test_file_path = tf.keras.utils.get_file("eval.csv", TEST_DATA_URL)

16384/13049 [=====================================] - 0s 0us/step


In [0]:
# Make numpy values easier to read - round to 3 decimal places, suppress trailing digits
np.set_printoptions(precision=3, suppress=True)

In [0]:
# Look at the head (top 10 lines) of the CSV file to see how it is formatted
!head {train_file_path}

survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.25,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
1,female,26.0,0,0,7.925,Third,unknown,Southampton,y
1,female,35.0,1,0,53.1,First,C,Southampton,n
0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y
0,male,2.0,3,1,21.075,Third,unknown,Southampton,n
1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n
1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n
1,female,4.0,1,1,16.7,Third,G,Southampton,n


This dataset can be read into a `pandas` `DataFrame` and then transformed into a `numpy` array which can then be loaded into a `tf` model.

However, for scalability or compatibility with `tf`, we can also use the `tensorflow.experimental.make_csv_dataset` function.

In [0]:
# Label is the only column we need to identify 
LABEL_COLUMN = 'survived'
LABELS = [0, 1]

In [0]:
# Reading the CSV datad
def get_dataset(file_path, **kwargs):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path, 
      batch_size=5,    # Artificially small to make samples easier to show
      label_name=LABEL_COLUMN,
      na_value="?",    # Missing values will be substituted with this
      num_epochs=1,    
      ignore_errors=True,
      **kwargs)
  
  return dataset

In [0]:
# Use this function to read in the raw data from the CSV files
raw_train_data = get_dataset(train_file_path)
raw_test_data = get_dataset(test_file_path)

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key, value.numpy()))

Each item in the dataset is a batch represented as tuple of multiple examples and labels. The data from the examples is organized into column-based (rather than row-based) tensors. Basically, instead of having one row in the tensor = one sample (multiple features), the dataset stores the samples as columns (in batches of `n` - `n` determined by us during init). 

In [19]:
show_batch(raw_train_data)

sex                 : [b'female' b'male' b'female' b'male' b'female']
age                 : [17. 28. 14. 28. 36.]
n_siblings_spouses  : [4 0 1 0 0]
parch               : [2 0 0 0 2]
fare                : [ 7.925 56.496 30.071  7.896 71.   ]
class               : [b'Third' b'Third' b'Second' b'Third' b'First']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'B']
embark_town         : [b'Southampton' b'Southampton' b'Cherbourg' b'Cherbourg' b'Southampton']
alone               : [b'n' b'y' b'n' b'y' b'n']


In [0]:
CSV_COLUMNS = ['survived', 'sex', 'age', 'n_siblings_spouses', 'parch', 'fare', 'class', 'deck', 'embark_town', 'alone']

In [0]:
# Initialise a temporary dataset with column names this time
temp_dataset = get_dataset(train_file_path, column_names=CSV_COLUMNS)

In [22]:
# Show batch will still show the same results because in the case of this dataset,
# the column names were in the first line, so were read automatically
show_batch(temp_dataset)

sex                 : [b'female' b'male' b'female' b'male' b'male']
age                 : [24. 30. 15. 17. 21.]
n_siblings_spouses  : [0 0 0 0 0]
parch               : [2 0 1 0 0]
fare                : [ 16.7    10.5   211.337   7.125   7.796]
class               : [b'Third' b'Second' b'First' b'Third' b'Third']
deck                : [b'G' b'unknown' b'B' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Southampton'
 b'Southampton']
alone               : [b'n' b'y' b'n' b'y' b'y']


In [23]:
# Can also choose to select specific columns from the dataset
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'class', 'deck', 'alone']
temp_dataset = get_dataset(train_file_path, select_columns=SELECT_COLUMNS)
show_batch(temp_dataset)

age                 : [23. 17. 25. 45. 33.]
n_siblings_spouses  : [0 0 1 0 0]
class               : [b'Second' b'Third' b'Second' b'Third' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
alone               : [b'y' b'y' b'n' b'y' b'y']


## Data Preprocessing
CSV may contain a variety of data types. Ideally, we would want to convert to these types into a single, fixed length vector. `tf.feature_column` can be used for this, but we don't really need it here since the features we will use are all numeric anyway. 

In [0]:
SELECT_COLUMNS = ['survived', 'age', 'n_siblings_spouses', 'parch', 'fare']

# Default values to be substituted in place of missing data
# Only the label is an int 0 - we're assuming if no data is available on the passenger's
# survival status then that passenger died. 
# All others are floats because they will be inputs for the tf model
DEFAULTS = [0, 0.0, 0.0, 0.0, 0.0] 

In [0]:
temp_dataset = get_dataset(train_file_path, 
                          select_columns=SELECT_COLUMNS,
                          column_defaults=DEFAULTS)

In [26]:
show_batch(temp_dataset)

age                 : [23. 28.  7. 26. 25.]
n_siblings_spouses  : [1. 0. 4. 0. 1.]
parch               : [0. 0. 1. 0. 0.]
fare                : [113.275   7.75   39.688   7.896  91.079]


In [0]:
# Read in this batch from the `iterator` object
example_batch, labels_batch = next(iter(temp_dataset))

In [0]:
# A function to pack together all the feature columns for the NN input
def pack(features, label):
  return tf.stack(list(features.values()), axis=-1), label

In [31]:
# Apply this to every element of the dataset
packed_dataset = temp_dataset.map(pack)

In [32]:
# Checking that the packing operation was successful
for features, labels in packed_dataset.take(1):
  print(features.numpy())
  print() # empty line
  print(labels.numpy())

[[28.     0.     0.     7.55 ]
 [45.     1.     1.    26.25 ]
 [42.     1.     0.    52.   ]
 [27.     0.     0.     8.663]
 [ 5.     2.     1.    19.258]]

[0 1 0 1 1]


This was just to show that one way of dealing with mixed datasets is to separate out simple numeric fields. The `tf.feature_column` API can handle non-numeric features as well as numeric ones. However, this incurs some overhead and should be avoided unless necessary.

IRL, you would actually use a mixed dataset. 

In [33]:
show_batch(raw_train_data)

sex                 : [b'male' b'male' b'male' b'male' b'male']
age                 : [17. 18. 45. 28.  1.]
n_siblings_spouses  : [0 1 0 0 4]
parch               : [0 0 0 0 1]
fare                : [ 8.663  6.496  6.975 15.05  39.688]
class               : [b'Third' b'Third' b'Third' b'Second' b'Third']
deck                : [b'unknown' b'unknown' b'unknown' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Southampton' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'y' b'n' b'y' b'y' b'n']


In [0]:
example_batch, labels_batch = next(iter(temp_dataset))

In [0]:
# Define a more general preprocessor that selects a list of numeric features and packs them into a single column
class PackNumericFeatures(object):
  # Constructor which will take a list of feature names to extract
  def __init__(self, names):
    self.names = names
  
  def __call__(self, features, labels):
    # Extract a feature from the dataset based on a column name passed in init
    numeric_features = [features.pop(name) for name in self.names]
    
    # Cast all the values for this feature to 32-bit floating point values
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    
    # Create a column of these values in a numpy/tf stack structure
    numeric_features = tf.stack(numeric_features, axis=-1)
    
    # Map these values back onto the features array
    features['numeric'] = numeric_features
    
    # Return the modified features and their corresponding labels
    return features, labels

In [36]:
# Now defining a list of names of numeric features
NUMERIC_FEATURES = ['age', 'n_siblings_spouses', 'parch', 'fare']

# Use the Class to transform the training data for these features
packed_train_data = raw_train_data.map(
  PackNumericFeatures(NUMERIC_FEATURES))

# Do the same for the test set
packed_test_data = raw_test_data.map(
  PackNumericFeatures(NUMERIC_FEATURES))

In [37]:
# Now examine the first batch of training data
show_batch(packed_train_data)

sex                 : [b'male' b'male' b'male' b'female' b'female']
class               : [b'Second' b'Third' b'First' b'First' b'Second']
deck                : [b'unknown' b'unknown' b'B' b'unknown' b'unknown']
embark_town         : [b'Southampton' b'Cherbourg' b'Southampton' b'Cherbourg' b'Southampton']
alone               : [b'y' b'y' b'y' b'y' b'n']
numeric             : [[47.     0.     0.    15.   ]
 [35.     0.     0.     7.896]
 [28.     0.     0.     0.   ]
 [30.     0.     0.    31.   ]
 [24.     1.     2.    65.   ]]


In [0]:
# Read this batch and its labels into vars
example_batch, labels_batch = next(iter(packed_train_data))

## Data Normalization
Continuous data should always be normalized
- mean normalization
- range or standard deviation normalization

But why use `tf.dataset` if the point was to use `pandas`?

In [41]:
import pandas as pd

# Chaining together function calls that will read CSV data from the file path
# Then extract the columns that we know to be numeric using their names
# And then displays central tendency and quartile statistics about them
desc = pd.read_csv(train_file_path)[NUMERIC_FEATURES].describe()
desc

,age,n_siblings_spouses,parch,fare
count,627.000000,627.000000,627.000000,627.000000
mean,29.631308,0.545455,0.379585,34.385399
std,12.511818,1.151090,0.792999,54.597730
min,0.750000,0.000000,0.000000,0.000000
25%,23.000000,0.000000,0.000000,7.895800
50%,28.000000,0.000000,0.000000,15.045800
75%,35.000000,1.000000,0.000000,31.387500
max,80.000000,8.000000,5.000000,512.329200


In [0]:
# Get the mean and standard deviation of each numeric column
MEAN = np.array(desc.T['mean']) # transpose?
STD  = np.array(desc.T['std'])


In [0]:
# Use these values to normalize all numeric entries
def normalize_numeric_data(data, mean, std):
  # Center the data 
  return (data - mean) / std

In [44]:
normalizer = functools.partial(normalize_numeric_data, mean=MEAN, std=STD)
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalizer, 
                                                 shape=[len(NUMERIC_FEATURES)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(4,), default_value=None, dtype=tf.float32, normalizer_fn=functools.partial(<function normalize_numeric_data at 0x7f38c3fefea0>, mean=array([29.631,  0.545,  0.38 , 34.385]), std=array([12.512,  1.151,  0.793, 54.598])))

In [45]:
example_batch['numeric']

<tf.Tensor: id=729, shape=(5, 4), dtype=float32, numpy=
array([[28.   ,  0.   ,  0.   ,  7.896],
       [28.   ,  0.   ,  0.   ,  7.25 ],
       [28.   ,  0.   ,  0.   ,  6.95 ],
       [32.5  ,  1.   ,  0.   , 30.071],
       [40.   ,  0.   ,  0.   , 13.   ]], dtype=float32)>

In [46]:
# Creating a densely connected layer using these featueres
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-0.13 , -0.474, -0.479, -0.485],
       [-0.13 , -0.474, -0.479, -0.497],
       [-0.13 , -0.474, -0.479, -0.503],
       [ 0.229,  0.395, -0.479, -0.079],
       [ 0.829, -0.474, -0.479, -0.392]], dtype=float32)

## Categorical Data
Can use the `tf.feature_column` API to transform categorical features to numeric ones.

In [0]:
# Create a dictionary of categories - keys are the col names, vals are possible categorical values
CATEGORIES = {
    'sex': ['male', 'female'], 
    'class': ['First', 'Second', 'Third'], 
    'deck': ['A', 'B', 'C', 'D', 'E', 'F'], 
    'embark_town': ['Cherbourg', 'Southampton', 'Queenstown'],
    'alone': ['y', 'n']
}

In [0]:
# Initialise as empty list
categorical_columns = []

# For every feature and list of possible values in the categorical dictionary
for feature, vocab in CATEGORIES.items():
  # idk this is some API mumbo jumbo
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [49]:
# What did I just do?
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='sex', vocabulary_list=('male', 'female'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='class', vocabulary_list=('First', 'Second', 'Third'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='deck', vocabulary_list=('A', 'B', 'C', 'D', 'E', 'F'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='embark_town', vocabulary_list=('Cherbourg', 'Southampton', 'Queenstown'), dtype=tf.string, default_value=-1, num_oov_buckets=0)),
 IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='alone', vocabulary_list=('y', 'n'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

In [50]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy()[0])

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0.]


In [0]:
# Concatenate both the categorical and non-categorical features
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns + numeric_columns)

## Build The Model

In [0]:
model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [0]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [54]:
train_data = packed_train_data.shuffle(500)
test_data = packed_test_data
model.fit(train_data, epochs=20)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
126/126 [==============================] - 2s 16ms/step - loss: 0.5152 - accuracy: 0.7656
Epoch 2/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4227 - accuracy: 0.8214
Epoch 3/20
126/126 [==============================] - 0s 3ms/step - loss: 0.4066 - accuracy: 0.8214
Epoch 4/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3923 - accuracy: 0.8278
Epoch 5/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3805 - accuracy: 0.8485
Epoch 6/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3720 - accuracy: 0.8437
Epoch 7/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3668 - accuracy: 0.8485
Epoch 8/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3525 - accuracy: 0.8565
Epoch 9/20
126/126 [==============================] - 0s 3ms/step - loss: 0.3511 - accuracy: 0.8565
Epoch

In [55]:
test_loss, test_accuracy = model.evaluate(test_data)

53/53 [==============================] - 0s 9ms/step - loss: 0.4685 - accuracy: 0.8068


In [56]:
print("\n\nTest Loss{}, Test Accuracy {}".format(test_loss, test_accuracy))



Test Loss0.46849653462194046, Test Accuracy 0.8068181872367859


In [0]:
predictions = model.predict(test_data)

In [59]:
# Show results
for prediction, survived in zip(predictions[:10], list(test_data)[0][1][:10]):
  print("Predicted Survival: {:.2%}".format(prediction[0]),
       "| Actual outcome: ", 
       ("SURVIVED" if bool(survived) else "DIED"))

Predicted Survival: 99.95% | Actual outcome:  SURVIVED
Predicted Survival: 67.67% | Actual outcome:  DIED
Predicted Survival: 91.19% | Actual outcome:  SURVIVED
Predicted Survival: 91.31% | Actual outcome:  DIED
Predicted Survival: 1.41% | Actual outcome:  DIED
